## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [7]:
import datetime
import os

from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware

from notebooks import get_twap_bpt_price
from notebooks import get_block_by_ts
from notebooks.constants import ARB_CORE_POOLS
from notebooks.constants import MAINNET_CORE_POOLS
from notebooks.constants import POLYGON_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# arb_block_now = arb_web3.eth.block_number - 1000
timestamp_now = 1687478400
arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
# Given Arb block time, we want to look back 2 weeks:
arb_timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
arb_block_2_weeks_ago = get_block_by_ts(arb_timestamp_2_weeks_ago, "arbitrum")
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
# Given mainnet block time, we want to look back 2 weeks:
mainnet_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
mainnet_block_2_weeks_ago = get_block_by_ts(mainnet_timestamp_2_weeks_ago, "mainnet")
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
# Given polygon block time, we want to look back 2 weeks:
poly_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
poly_block_2_weeks_ago = get_block_by_ts(poly_timestamp_2_weeks_ago, "polygon")
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

print("ARBITRUM")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3, start_date=arb_datetime_now.date(),
                                                       block_number=arb_block)
    print(arb_bpt_twap_prices[arb_pool])

print("MAINNET")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now.date(),
                                                               block_number=mainnet_block)
    print(mainnet_bpt_twap_prices[mainnet_pool])

print("POLYGON")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now.date(), block_number=poly_block)
    print(polygon_bpt_twap_prices[polygon_pool])


ARBITRUM
Pool wasn't created at the block number
None
1754.597908209454420757718353
Pool wasn't created at the block number
None
Pool wasn't created at the block number
None
1.264576957037256764606735381
Pool wasn't created at the block number
None
Pool wasn't created at the block number
None
MAINNET
Pool wasn't created at the block number
None
1785.910058109186289841243072
Pool wasn't created at the block number
None
Pool wasn't created at the block number
None
Pool wasn't created at the block number
None
30.09523350749633653916419923
1751.025369754485812445012438
Pool wasn't created at the block number
None
1746.106481685917550356867449
59.32538906603669817662975698
62.11761019120373965328023306
276.6688493095790426885881111
0.7522628056713356709384671248
1748.248715749615811213783619
9.377604724756727074175137664
POLYGON
Pool wasn't created at the block number
None
Pool wasn't created at the block number
None
Pool wasn't created at the block number
None
Pool wasn't created at the bl

In [8]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
      totalProtocolFeePaidInBPT
      tokens {{
        symbol
        address
        paidProtocolFees
      }}
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [9]:
from notebooks.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.constants import ARB_BALANCER_GRAPH_URL
from notebooks.constants import MAINNET_BALANCER_GRAPH_URL
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

## Extract fee data from CORE pools:


In [10]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.date) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date) or 0
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                      token_fees[pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now.date())
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now.date())
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now.date())
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [11]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
B-rETH-STABLE,25208.10,3052.45,0.00,0.00,25130.13,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
50rETH-50BADGER,4280.64,1770.46,69.14,4294.69,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
B-staFiETH-WETH-Stable,3329.86,2490.60,0.00,0.00,3299.51,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
wbETH-wstETH,3006.23,50.13,1.62,2837.08,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
50rETH-50RPL,2851.95,2846.81,10.67,2951.02,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
20WETH-80ALCX,2134.51,3384.73,57.58,1732.97,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
50wstETH-50ACX,1646.27,1236.54,174.54,1636.80,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
ankrETH/wstETH,1597.73,877.25,0.00,0.00,0.00,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
50wstETH-50LDO,624.67,939.73,10.34,613.33,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet
50STG-50bbaUSD,304.92,18668.04,385.06,289.67,0,2023-06-09 03:00:00,2023-06-23 03:00:00,mainnet


## Now let's calculate bribes paid to the pools

In [12]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees
# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str) -> Dict[str, Dict]:
    pool_bribs = {}
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": pool_fees * FEE * aura_vebal_share,
            "bal_bribes": pool_fees * FEE * (1 - aura_vebal_share),
            "pool_total": pool_fees,
            "fees_taken": pool_fees * FEE,
        }
    return pool_bribs
mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet')
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum')
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon')
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'../data/bribs/bribes_{arb_datetime_2_weeks_ago.date()}_{arb_datetime_now.date()}.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_total,fees_taken
B-wstETH-WETH-Stable,arbitrum,711.3138254616805087310042276,1477.991174538319491268995772,4378.61,2189.305
RDNT-WETH,arbitrum,4650.400486673158688234721437,9662.754513326841311765278563,28626.31,14313.155
B-rETH-STABLE,mainnet,4082.439154126387415841162211,8482.625845873612584158837789,25130.13,12565.065
20WETH-80ALCX,mainnet,281.5243924693746351503258788,584.9606075306253648496741212,1732.97,866.485
B-staFiETH-WETH-Stable,mainnet,536.0119033778001364275502406,1113.743096622199863572449759,3299.51,1649.755
ankrETH/wstETH,mainnet,0E-31,0E-31,0.00,0.000
50wstETH-50LDO,mainnet,99.63666747447534866544104703,207.0283325255246513345589530,613.33,306.665
50rETH-50BADGER,mainnet,697.6808560415347939282001700,1449.664143958465206071799830,4294.69,2147.345
50rETH-50RPL,mainnet,479.3990159465968457741995966,996.1109840534031542258004034,2951.02,1475.510
50STG-50bbaUSD,mainnet,47.05746248729276938665695155,97.77753751270723061334304845,289.67,144.835
